# import libraries

In [1]:
#--- system libraries
import pdb
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import traceback
import os
import scipy.interpolate as scp_int
import warnings
import matplotlib
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import patches
import sys
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
import patsy
#
warnings.filterwarnings('ignore')

#--- user modules
import LammpsPostProcess as lp
import imp
imp.reload(lp)

<module 'LammpsPostProcess' from '/Users/Home/Desktop/Tmp/txt/git/CrystalPlasticity/postprocess/LammpsPostProcess.py'>

# Parameters

In [4]:
#--- comment out if no parameter is passed as an env. variable
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

path = os.environ.get('path')

ValueError: not enough values to unpack (expected 2, got 1)

In [5]:
#--- comment out parameters if you pass them
#--- by arguments

#--- timestep
dlgr0 = 2.5 #--- 1st peak  in g(r)
dlgr = 4.0 #--- 2nd peak in g(r) !!!!!!!!!!
cutoff = 10.0*4    #--- cut-off radius for g(r)
NMAX = 10000*4    #--- max. number of atoms included for the g(r) analysis
#
ReadStrsFromDump = False #--- if true: read stress from dump file
#
#--- if np.any(AssignYieldTime,AssignMaxDropTime,AssignFlowTime) -> comment itime 
AssignYieldTime = False #--- stress peak
AssignMaxDropTime = False #--- max drop
AssignFlowTime = False #--- flow stress
#
ShuffleDisp = False
CrltnFunctionUnstructured = False
PairCorrelationFunction = False
#
CommonNeighAnl = True #--- Read ovito output
#
StrnAnalysis = False
#
DminAnalysis = False
#
RemoteMachine = True #--- tex issue: don't run corresponding cells
# 'Co', 'Ni', 'Cr', 'Fe', 'Mn'
#LatticeConstant = {1:58.933,2:58.690,3:51.960,4:55.847,5:54.940} #--- from library_CoNiCrFeMn.meam
AtomicRadius = {1:1.35,2:1.35,3:1.4,4:1.4,5:1.4}

#--- lammps output
#path = '/mnt/home/kkarimi/Project/git/CrystalPlasticity/testRuns/glassFeNi/Run0'
#path='/Users/Home/Desktop/Tmp/txt/git/CrystalPlasticity/BmgData'
path='/Users/Home/Desktop/Tmp/txt'

#--- bulk stress files
indx = 0
fileNamee = ('FeNi.txt', 
             'CoNiFe.txt', 
             'CoNiCrFe.txt',
             'CoCrFeMn.txt',
             'CoNiCrFeMn.txt',
             'Co5Cr5Fe40Mn27Ni26.txt')[indx]

#--- xyz and atom-wise stress files
fileName = '%s/%s'%(path,[
            'FeNi_glass.dump',
            'CoNiFe_glass.dump',
            'CoNiCrFe_glass.dump',
            'CoCrFeMn_glass.dump',
            'CoNiCrFeMn_glass.dump',
            'Co5Cr5Fe40Mn27Ni26_glass.dump'][indx])#--- parse cords

fileName2nd = '%s/%s'%(path,[
            'FeNi_Cna.xyz',
            'CoNiFe_Cna.dump',
            'CoNiCrFe_Cna.dump',
            'CoCrFeMn_Cna.dump',
            'CoNiCrFeMn_Cna.dump',
            'Co5Cr5Fe40Mn27Ni26_Cna.dump'][indx])#--- parse cords


In [ ]:
!ovitos OvitosCna.py $fileName d2min.xyz $nevery 2